In [17]:
import os
import glob
import numpy as np
import cv2
import torch
import warnings

In [18]:
from utils.xyz import gamma, positional_encoder
from utils.nets import Nerf

In [19]:
vec = torch.rand(10,5)
model = Nerf()
sigma, color = model.forward(vec)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x274 and 283x128)

In [ ]:
from utils.dataload import load_data
data, cam_params = load_data('data/nerf_synthetic/lego/')

In [ ]:
from utils.xyz import rays_single_cam

def rays_dataset(samples, cam_params):
	""" Generates rays and camera origins for train test and val sets under diff camera poses""" 
	keys = ['train', 'test', 'val']
	rays_1_cam = rays_single_cam(cam_params)
	rays = {}
	cam_origins = {}
	H, W, f = cam_params
	for k in keys:
		num_images = len(samples[k])
		transf_mats = torch.stack([s['transform'] for s in samples[k]])
		rays_dataset = torch.matmul(transf_mats[:,:3,:3], rays_1_cam)
		cam_origins = transf_mats[:,:3,3:]
		cam_origins = cam_origins.expand(num_images,3,H*W) #Nx3xHW
		rays[k] = torch.cat((cam_origins, rays_dataset),dim=1).permute(1,0,2).reshape(6,-1) # 6xNHW, number of cameras 

	return rays

class RayGenerator:
	def __init__(self, path, on_gpu=False):
		samples, cam_params = load_data(path)
		self.samples = samples
		self.H = cam_params[0]
		self.W = cam_params[1]
		self.f = cam_params[2]
		self.rays_dataset = rays_dataset(self.samples, cam_params)
		
		# self.on_gpu = on_gpu
		# if on_gpu:
		# 	for k in self.cam_origins.keys():
		# 		self.rays_dataset[k] = self.rays_dataset[k].cuda()

	def select(self, mode='train', N=4096):
		""" randomly selects N train/test/val rays
		Args:
			mode: 'train', 'test', 'val'
			N: number of rays to sample 
		Returns:
			rays (torch Tensor): Nx6 
		"""
		data = self.rays_dataset[mode]
		samples = self.samples[mode]
		ray_ids = torch.randint(0, data.size(1), (N,))
		rays = data[:,ray_ids]
		return rays, ray_ids 
		
rays = rays_dataset(data, cam_params)



def sampler(rays_dataset, batch_size):
	""" create a batch of rays from rays_dataset 
	Args:
		rays_dataset (torch tensor): Nx3xHW . N - #cameras, HW - #pixels in image
		batch_size: batch size of rays 
	"""

In [ ]:
from utils.dataload import rays_dataset, RayGenerator
from utils.nets import Nerf
import torch

rg = RayGenerator(path='data/nerf_synthetic/lego/')

net = Nerf()
net.load_state_dict(torch.load('models/1665717276.4336596.pth'))

samps = rg.samples['val']


In [ ]:
import matplotlib.pyplot as plt
from utils.rendering import render_nerf 
from tqdm import tqdm 

plt.imshow(rg.samples['val'][0]['img'])

NUM_IMG_RAYS = 640000
net = net.cuda()
rays = rg.rays_dataset['val'][:,:NUM_IMG_RAYS].transpose(1,0)

batch_size = 64000

rgbs = [] 
depths = [] 
with torch.no_grad():
	for i in tqdm(range(rays.size(0) // batch_size)):
		inp_rays = rays[i*batch_size:(i+1)*batch_size]
		rgb, depth, _, _, _ = render_nerf(inp_rays.cuda(), net, N=128)
		rgbs.append(rgb)
		depths.append(depth)

rgb = torch.cat(rgbs)
depth = torch.cat(depths)

print(rgb.size())
print(depth.size())



In [ ]:
rgb_img = rgb.reshape(800,800,3)

depth_img = depth.reshape(800,800).cpu().numpy()
plt.imshow(depth_img)
plt.figure()
plt.imshow(rgb_img.cpu().numpy())

In [ ]:
rays.size(0)

In [ ]:
# rays, ray_ids = rg.select()
# train_imgs = torch.stack([torch.from_numpy(s['img']) for s in rg.samples['train']]).reshape(-1,3).transpose(1,0)


import torch 

N = 4096
ray_ids = torch.randint(0, 16000000, (N,))

print(torch.unique(ray_ids).size())

In [ ]:
torch.randperm(10)

In [ ]:
def render_nerf(rays, N, tn=2, tf=6):
	""" stratified sampling on a set of rays using Nerf model
	Args:
		rays (torch Tensor): B x 6 
		net (nerf model):
	Returns:
		 out: BxNx4	
	"""
	## input to nerf model BN x 6 
	## Nerf output - BN x 4  --> reshape BxNx4 
	## t sample dims - BxN
	t_bins = torch.linspace(tn,tf,N+1)
	bin_diff = t_bins[1] - t_bins[0] 

	unif_samps = torch.rand(rays.size(0),N)
	ts = bin_diff* unif_samps + t_bins[:-1] # BxN 

	origins = rays[:,:3] # Bx3
	dirs = rays[:,3:]  # Bx3

	disp = dirs.unsqueeze(-1)*ts.unsqueeze(1) # Bx1x3 * BxNx1 = Bx3xN
	locs = origins.unsqueeze(-1) + disp # Bx3x1 + Bx3xN = Bx3xN 
	query_pts = torch.cat((locs, dirs.unsqueeze(-1).expand(-1,-1,N)),dim=1)
	return query_pts 

query_pts = render_nerf(rays, N=100)

query_pts[0][:,:8]

In [ ]:
def volume_render(ts, nerf_outs):
	""" computes color, depth and alphas along rays using NeRF outputs (section 4)
	Args:
		ts (torch tensor) BxN | B:number of rays, N: number of samples along a ray 
		nerf_outs (torch tensor)BxNx4 | RGB \sigma (4 values) for each sample along each ray 
	Returns: 
		rgb (torch tensor) Bx3
		depth (torch tensor) (B,)
		alphas (torch tensor) (B,N)
		acc (torch tensor) (B,)
		w (torch tensor) (B,N) 
	"""

	deltas = ts[:,1:] - ts[:,:-1]
	deltas = torch.cat((deltas, 1e10*torch.ones_like(deltas[:,:1])), dim=1)
	## TODO add gaussian noise regularizer 

	alpha = 1 - torch.exp(-nerf_outs[...,3]*deltas)
	T = torch.exp(-torch.cumprod(nerf_outs[...,3]*deltas, dim=1))
	
	## Eqn 5 of paper w: BxN
	w = alpha*T 
	
	## accumulated rgb color along each ray (Bx3)
	rgb = torch.sum(w.unsqueeze(-1)*nerf_outs[...,:3], axis=1)

	## depth along each ray (B,), weighted average of samples 
	depth = torch.sum(w * ts, axis=-1)

	print(depth.size())
	## accumulation map (B,): average of weight values along a ray 
	acc = torch.sum(w, axis=-1)
	rgb += 1-acc.unsqueeze(-1)
	return rgb, depth, alpha, acc, w
ts = torch.rand(5,10)
nerf_outs = torch.rand(5,10,4)


rgb, depth, alpha, acc, w = volume_render(ts, nerf_outs)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.imshow(data['test'][50]['img'])

print(cam_params)

In [ ]:
np.max(data['train'][0]['img'])
	

In [ ]:
import torch 

H , W = 800, 800
f = 1100

Hl = torch.arange(H) - H//2
Wl = torch.arange(W) - W//2
grid_x, grid_y = torch.meshgrid(Hl, Wl)
ray_matrix = torch.stack((grid_x, grid_y, f*torch.ones_like(grid_x))).float()

ray_matrix = ray_matrix / torch.norm(ray_matrix, dim=0)
ray_matrix = torch.reshape(ray_matrix, (3,-1)) # 640K ray directions, normalized

In [ ]:
import visu3d as v3d 
import numpy as np

In [ ]:
import numpy as np
import torch 
from utils.xyz import rays_single_cam
from utils.dataload import RayGenerator

rg = RayGenerator('data/nerf_synthetic/lego')

rays = rg.rays_dataset['train']
f = 1111 
H = 800
W = 800
rays_cam = rg.rays_dataset['train'].numpy().transpose(1,0)
B = 1

rays = v3d.Ray(pos=rays_cam[:,:3][np.newaxis,...], dir=5*rays_cam[:,3:][np.newaxis,...])

print(rays.shape)
# ray = v3d.Ray(pos=[0,0,0], dir=[1,1,1])
# rays = v3d.Ray(pos=np.zeros(640000,), dir=np.ones((B,H,W,3)))
# assert rays.shape == (B,H,W)

In [ ]:
N = 640000
i = 8
rays = v3d.Ray(pos=rays_cam[:,:3][np.newaxis,...], dir=4*rays_cam[:,3:][np.newaxis,...])

In [ ]:
import torch
from utils.nets import Nerf
# from utils.rendering import volume_render
tn, tf = 2, 6 
N = 100
net = Nerf()
net.load_state_dict(torch.load('models/1666132529.4454966.pth'))

volume_rep = torch.zeros(1,N,4)
volume_rep[:,:,:3]= torch.tensor([1.0,0.,0.])
volume_rep[:,1,3:] = 100
volume_rep[:,0,3:] = 0.0
volume_rep = volume_rep.cuda()

B = 1

t_bins = torch.linspace(tn,tf,N+1)
bin_diff = t_bins[1] - t_bins[0] 

unif_samps = torch.rand(B,N)
ts = bin_diff* unif_samps + t_bins[:-1]
ts = ts.cuda()


rgb, depth, alpha, acc, w = volume_render(volume_rep, ts)

print(depth)
print(rgb)
# print(ts)
# print(alpha)
# print(ts)
# print(w)
# print(rgb, depth, alpha, acc, w)


In [ ]:
def volume_render(nerf_outs, ts):
	""" computes color, depth and alphas along rays using NeRF outputs (section 4)
	Args:
		ts (torch tensor) BxN | B:number of rays, N: number of samples along a ray 
		nerf_outs (torch tensor)BxNx4 | RGB \sigma (4 values) for each sample along each ray 
	Returns: 
		rgb (torch tensor) Bx3
		depth (torch tensor) (B,)
		alphas (torch tensor) (B,N)
		acc (torch tensor) (B,)
		weights (torch tensor) (B,N) 
	"""
	deltas = ts[:,1:] - ts[:,:-1]
	deltas = torch.cat((deltas, 1e10*torch.ones_like(deltas[:,:1])), dim=1)
	## TODO add gaussian noise regularizer 

	alpha = 1 - torch.exp(-nerf_outs[...,3]*deltas)
	T = torch.exp(-torch.cumprod(nerf_outs[...,3]*deltas, dim=1))
	
	## Eqn 5 of paper w: BxN 
	weights = alpha*T 
	
	## accumulated rgb color along each ray (Bx3) 

	# print(weights.unsqueeze(-1)*nerf_outs[...,:3])
	rgb = torch.sum(weights.unsqueeze(-1)*nerf_outs[...,:3], axis=1)
	# print(rgb)
	## depth along each ray (B,), weighted average of samples 
	depth = torch.sum(weights * ts, axis=-1)
	# print(depth)
	## accumulation map (B,): average of weight values along a ray 
	acc = torch.sum(weights, axis=-1)
	# print(acc)
	# rgb += 1-acc.unsqueeze(-1)

	return rgb, depth, alpha, acc, weights

In [ ]:
def render_image(net, rg, batch_size=64000, im_idx=0, im_set='val'):
	""" render an image and depth map from val set (currently hardcoded) from trained NeRF model """

	gt_img = rg.samples[im_set][im_idx]['img']
	NUM_RAYS = 640000 # number of rays in image, currently hardcoded
	net = net.cuda()
	rays = rg.rays_dataset[im_set][:,im_idx*NUM_RAYS:(im_idx+1)*NUM_RAYS].transpose(1,0)
	# rays = rg.rays_dataset[im_set][im_idx*NUM_RAYS:(im_idx+1)*NUM_RAYS,:]
	rgbs = [] 
	depths = [] 
	with torch.no_grad():
		for i in tqdm(range(rays.size(0) // batch_size)):
			inp_rays = rays[i*batch_size:(i+1)*batch_size]
			rgb, depth, _, _, _ = render_nerf(inp_rays.cuda(), net, N=128)
			rgbs.append(rgb)
			depths.append(depth)

	rgb = torch.cat(rgbs).cpu()
	depth = torch.cat(depths).cpu()
	
	rgb_img = rgb.reshape(1,800,800,3) ## permuting for tensorboard
	depth_img = depth.reshape(1,800,800,1) ## permuting for tensorboard
	gt_img = gt_img.reshape(1,800,800,3)
	return rgb_img, depth_img, gt_img

In [ ]:
import numpy as np
lr_start = 5e-4
lr_end = 5e-5
decay = np.exp(np.log(lr_end / lr_start) / 10000)

In [ ]:
torch.clip(torch.tensor(decay), torch.tensor(0), torch.tensor(1))

In [ ]:
# Ray helpers
import numpy as np
import torch
def get_rays(H, W, K, c2w):
    i, j = torch.meshgrid(torch.linspace(0, W-1, W), torch.linspace(0, H-1, H))  # pytorch's meshgrid has indexing='ij'
    i = i.t()
    j = j.t()
    dirs = torch.stack([(i-K[0][2])/K[0][0], -(j-K[1][2])/K[1][1], -torch.ones_like(i)], -1)
    # Rotate ray directions from camera frame to the world frame
    rays_d = torch.sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)  # dot product, equals to: [c2w.dot(dir) for dir in dirs]
    # Translate camera frame's origin to the world frame. It is the origin of all rays.
    rays_o = c2w[:3,-1].expand(rays_d.shape)
    return rays_o, rays_d
    

In [ ]:
# from utils.xyz import rays_single_cam
H = 2
W = 2
f = 10
K = np.array([[f,0,H//2],[0,f,W//2],[0,0,1]])

c2w = torch.eye(4)
ro, rd = get_rays(H, W, K, c2w)
rd_ = rays_single_cam([H,W, f])

print(rd_.shape)
print(rd.reshape(-1,3) - rd_.T)

In [ ]:
def rays_single_cam(cam_params):
	""" takes in camera params H,W,f returns H*W ray directions with origin 0,0,0
	Args:
		cam_params (list): [H, W, f]
	Returns:
		rays (torch Tensor): 3 x HW
	"""
	H , W, f  = cam_params
	Hl = torch.arange(H) - H//2
	Wl = torch.arange(W) - W//2
	print(Hl)
	grid_x, grid_y = torch.meshgrid(Wl, Hl)
	print(grid_x, grid_y)
	rays = torch.stack((grid_x/f, -grid_y/f, -1*torch.ones_like(grid_x))).float()
	rays = rays.permute(0,2,1)
	# rays = rays / torch.norm(rays, dim=0)
	rays = torch.reshape(rays, (3,-1)) # 640K ray directions (if H,W = 800), normalized
	return rays


In [ ]:
import cv2 
import matplotlib.pyplot as plt
%matplotlib widget
idxs = [0, 5, 50, 58, 65]

ims = []
for idx in idxs:
	path = f'results/lego/mynerf_25_imgs/rgb_{idx}.png'
	im = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
	print(im.shape)
	ims.append(im[:,403:,:])

im_big = np.hstack(ims)

print(im_big.shape)

plt.imshow(im_big)

In [ ]:
W = 400
x = np.random.choice(np.arange(400), size=(400,), replace=False)
y = np.random.choice(np.arange(400), size=(400,), replace=False)
xv, yv = np.meshgrid(np.arange(400), np.arange(400), indexing='ij')

In [20]:
import visu3d as v3d
import numpy as np
def samp_sphere(center, radius, num=1000):
	"""sample num points on surface of sphere"""
	pass 
	## use this for visualization 

def generate_phaseop_params(H,W,f, num):
	""" create phase optic parameters """
	pass

def refract(rays, n, mu):
	""" returns refracted ray incident at surface with surface normal n and refractive index mu w.r.t air
	Args:
		rays: (nx3) Incident ray directions
		n: (nx3) normals at the points where rays hit surface 
		mu: (scalar) refractive index of air w.r.t surface
	Returns:
		t (nx3) : refracted rays
	"""
	i_norms = np.linalg.norm(rays, axis=1, keepdims=True)
	i = rays / i_norms
	n = n / np.linalg.norm(n,axis=1, keepdims=True) 
	ni = np.sum(n*i, axis=1,keepdims=True)
	t = mu*i + n*np.sqrt(1 - (mu**2)*(1 - ni**2)) - mu*n*ni
	t = t * i_norms
	return t 
	

def intersect_plane(rays, normal, point):
	""" computes intersection of rays with plane 
	Args:
		rays (Nx6 np.array): origin and direction of rays 
		normal (3, np.array): normal vector to the plane 
		point (3, np.array): a point that lies on the plane 
	Returns:
		Nx6 rays with new origin on plane and same direction as before
	""" 
	## TODO : Handle the edge case for ray parallel to plane
	ray_dirn = rays[:,3:]
	ray_origin = rays[:,:3]
	t = (normal.dot(point) - ray_origin.dot(normal))/ray_dirn.dot(normal)
	rays_out = np.concatenate((ray_origin + t.reshape(-1,1)*ray_dirn, ray_dirn),axis=1)
	return rays_out
	

def intersect_sphere(rays, center, radius):
	""" computes intersection of rays with sphere given by center and radius 
	Args:
		rays (Nx6 np.array): origin and direction of rays 
		center (3, np.array): center of sphere
		radius (scalar float): radius of sphere
	Returns:
		out (Nx6, Np.array): origin is same as input for rays that don't intersect
		valid: indices of rays which don't intersect with this sphere
	""" 
	rays_origin = rays[:,:3]
	rays_direction = rays[:,3:]
	rays_dirn = rays_direction / np.linalg.norm(rays_direction, axis=1, keepdims=True)
	b = 2 * np.sum(rays_dirn* (rays_origin - center.reshape(-1,3)), axis=1, keepdims=True)
	c = np.linalg.norm(rays_origin - center, axis=1, keepdims=True) ** 2 - radius ** 2
	delta = b ** 2 - 4 * c
	out = np.ones_like(rays_origin)
	t = (-b + np.sqrt(delta)) / 2
	v1 = delta > 0 
	v2 = t >=0
	valid = np.where(v1*v2 == True)[0]
	invalid = np.where(v1*v2 == False)[0]
	out[valid] = rays_origin[valid] + t[valid]*rays_dirn[valid]
	out[invalid] = rays_origin[invalid]
	rays_out = np.concatenate((out, rays_direction), axis=1)
	return rays_out, valid 

def intersect_sphere_batch(rays, centers, radii):
	""" compute intersection of rays with a batch of spheres 
	Args:
		rays (Nx6 np.array): origin and direction of rays 
		centers (Mx3, np.array): center of sphere
		radii (M, scalar float): radius of sphere
	Returns:
		out (Nx6, Np.array): origin is same as input for rays that don't intersect
		valid: indices of rays which don't intersect with this sphere
	""" 
	rays_origin = rays[:,:3]
	rays_direction = rays[:,3:]
	rays_dirn = rays_direction / np.linalg.norm(rays_direction, axis=1, keepdims=True)

	## reshaping for batching 
	N,_ = rays_origin.shape
	M = centers.shape[0]
	rays_origin = rays_origin.reshape(1,N,3)
	rays_dirn = rays_dirn.reshape(1,N,3)
	centers = centers.reshape(M,1,3)
	radii = radii.reshape(M,1,1)
	
	b = 2 * np.sum(rays_dirn * (rays_origin - centers), axis=2, keepdims=True)
	c = np.linalg.norm(rays_origin - centers, axis=2, keepdims=True) ** 2 - radii ** 2
	delta = b ** 2 - 4 * c

	rays_origin = rays_origin.squeeze(0)
	rays_dirn = rays_dirn.squeeze(0)

	out = np.ones_like(rays_origin)
	t = (-b + np.sqrt(delta)) / 2 # M,N,1
	v1 = delta > 0  # M,N,1
	v2 = t >=0 
	valid_int = v1*v2 
	t[~valid_int] = -np.Inf
	tmax = np.max(t, axis=0) # N,1 
	sphere_idx = np.argmax(t, axis=0).squeeze(-1) # N,1
	valid = np.where(tmax != -np.Inf)[0]
	invalid = np.where(tmax == np.Inf)[0]

	out[valid] = rays_origin[valid] + tmax[valid]*rays_dirn[valid]
	out[invalid] = rays_origin[invalid]
	rays_out = np.concatenate((out, rays_direction), axis=1)
	return rays_out, valid, sphere_idx   


def visu3d_tracer(rays_list):
	""" creates a set of visual rays to be traced from list of Nx6 rays
	Args:
		rays_list (list [r0, r1, ... rn]): r0 - Nx6. 0:3 origin, 3: direction 
		r_i origin = end point of r_(i-1). Assumes r_i and r_(i-1) have diff origins 
	Returns rt - Nx6 : visu3d compatible rays for tracing 
	"""
	rt = []
	for idx, r in enumerate(rays_list[:-1]):
		o, d = r[:,:3] , r[:,3:]
		r1 = rays_list[idx+1]
		o1, d1 = r1[:,:3], r1[:,:3] 
		dist = np.linalg.norm(o1 - o, axis=1, keepdims=True)
		d = d * dist / np.linalg.norm(d,axis=1, keepdims=True)
		r[:,3:] = d
		rt.append(r)
	rt.append(rays_list[-1])
	rt = np.concatenate(rt,axis=0)
	poss = rt[:,:3]
	dirss = rt[:,3:]
	viz = v3d.Ray(pos=poss, dir=dirss)
	return viz

class SinglePhaseOptic:
	def __init__(self, center, radius, n=1.5):
		self.center = center
		self.radius = radius 

class PhaseOptic:
	def __init__(self, centers, radii, n=1.5):
		self.centers = centers
		self.radii = radii 
		self.n = n # refractive index
	def visualize(self):
		""" function to visualize profile of phase optic """
	

In [ ]:
""" phase optic ray sphere intersection algorithm 
For every point 
1. Calculate intersection of a ray with all spheres.
2. Create an array of all valid intersections, i.e. on right side of focal plane. 
3. Select the intersection with the maximum distance from the plane of phase optic. 
"""

""" 
Assumptions to create phase optic 
1. no sphere lies completely on 1 side of phase optic
2. radii of spheres in a range (rmin to rmax)
3. centers of spheres in a range 
"""

In [22]:
def sphere_intersect(center, radius, ray_origin, ray_direction):
	ray_direction = ray_direction / np.linalg.norm(ray_direction)
	b = 2 * np.dot(ray_direction, ray_origin - center)
	c = np.linalg.norm(ray_origin - center) ** 2 - radius ** 2
	delta = b ** 2 - 4 * c
	if delta > 0:
		t1 = (-b + np.sqrt(delta)) / 2
		t2 = (-b - np.sqrt(delta)) / 2
		t = max(t1, t2)
		print(t)
		if t > 0:
			return t, ray_origin + ray_direction*t
	return None, None

In [48]:
centers = np.array([[-0.2, 0., -1.],[0.2,0.0,-1.], [0., -0.2, -1.], [0., 0.2, -1.]])
radii = np.array([0.2,0.2,0.2,0.2])
print(centers.shape)
print(radii.shape)

(4, 3)
(4,)


In [49]:
import numpy as np
import visu3d as v3d
from utils.xyz import rays_single_cam
H,W,f = 400,400,1111

sphere_center = np.array([0.1, 0.2, -0.8])

rays = rays_single_cam([H,W,f]).T.numpy()
rays_inp = np.concatenate((np.zeros_like(rays), rays),axis=1)
r0 = rays_inp
rays_inp = intersect_plane(rays_inp, np.array([0., 0., -1.]), np.array([0., 0., -1.]))
r1=rays_inp
plane_normals = np.zeros_like(rays)
plane_normals[:,2] = -1
rays_refrac = refract(rays_inp[:,3:], plane_normals, 1/1.5)

rays_inp = np.concatenate((rays_inp[:,:3], rays_refrac), axis=1)
r2 = rays_inp
rays_inp, valid = intersect_sphere(rays_inp, center=sphere_center, radius=0.3)
r3 = rays_inp

sphere_normals = rays_inp[:,:3] - sphere_center
sphere_normals = sphere_normals / np.linalg.norm(sphere_normals,axis=1,keepdims=True)

rays_refrac = refract(rays_inp[valid][:,3:], sphere_normals[valid], 1.5)
rays_inp[valid,3:] = rays_refrac
# rays_inp = np.concatenate((rays_inp[:,:3], rays_refrac), axis=1) 
r4 = rays_inp 

### Testing Multiple Lenslets
rays_inp_batch, valid_batch, sphere_idx = intersect_sphere_batch(rays_inp, centers, radii)
r3_batch = rays_inp_batch 

sphere_normals_batch = rays_inp_batch[:, :3] - centers[sphere_idx]
sphere_normals = sphere_normals / np.linalg.norm(sphere_normals,axis=1,keepdims=True)

rays_refrac_batch = refract(rays_inp_batch[valid_batch][:, 3:], sphere_normals_batch[valid_batch], 1.5)
rays_inp_batch[valid_batch,3:] = rays_refrac_batch
r4_batch = rays_inp_batch

/tmp/ipykernel_1668641/225820394.py:64: RuntimeWarning:

invalid value encountered in sqrt

/tmp/ipykernel_1668641/225820394.py:25: RuntimeWarning:

invalid value encountered in sqrt

/tmp/ipykernel_1668641/225820394.py:104: RuntimeWarning:

invalid value encountered in sqrt



In [51]:
import visu3d as v3d 

idx = 5
# poss = np.concatenate((r0[valid][::idx, :3], r1[::idx, :3]),axis=0)
# dirss = np.concatenate((r0[valid][::idx, 3:], r1[::idx, 3:]),axis=0)
viz = visu3d_tracer([r0[valid_batch][::idx], r2[valid_batch][::idx], r4_batch[valid_batch][::idx]])
viz.fig

In [392]:
import numpy as np 


a = np.random.randn(3,2,1)
b = np.random.randn(3,1,1)
print(a)
print(b)

print((a - b).shape)

[[[ 0.88099026]
  [ 0.51693184]]

 [[-0.06530861]
  [ 0.85787554]]

 [[ 0.49026553]
  [-0.20317115]]]
[[[-1.42682548]]

 [[-0.7708823 ]]

 [[ 1.25003083]]]
(3, 2, 1)


In [378]:
## check sphere separately 
H,W,f = 400,400,1111

sphere_center = np.array([0.1, 0.1, -1.])

rays = rays_single_cam([H,W, f]).T.numpy()
rays_inp = np.concatenate((np.zeros_like(rays), rays),axis=1)
r0 = rays_inp
rays_inp, valid = intersect_sphere(rays_inp, center=sphere_center, radius=0.1)
sphere_normals = rays_inp[valid][:,:3]- sphere_center
sphere_normals = sphere_normals / np.linalg.norm(sphere_normals,axis=1,keepdims=True)
rays_refrac = refract(rays_inp[valid][:,3:], sphere_normals, 1.5)
rays_inp = np.concatenate((rays_inp[valid][:,:3], rays_refrac), axis=1) 
r1 = rays_inp

/tmp/ipykernel_1639677/529445765.py:65: RuntimeWarning:

invalid value encountered in sqrt

/tmp/ipykernel_1639677/529445765.py:24: RuntimeWarning:

invalid value encountered in sqrt



In [379]:
import visu3d as v3d 

idx = 2000

# poss = np.concatenate((r0[valid][::idx, :3], r1[::idx, :3]),axis=0)
# dirss = np.concatenate((r0[valid][::idx, 3:], r1[::idx, 3:]),axis=0)

viz = visu3d_tracer([r0[valid][::idx], r1[::idx]])
viz.fig
# poss = r1[::idx, :3]
# dirss = r1[::idx, 3:]

# poss = np.repeat(sphere_center.reshape(1,3), 160000, 0)
# dirss = sphere_normals
# rays = v3d.Ray(pos=poss, dir=dirss)
# rays.fig

In [316]:
r0

array([[ 0.        ,  0.        ,  0.        , -0.18001801,  0.18001801,
        -1.        ],
       [ 0.        ,  0.        ,  0.        , -0.17911792,  0.18001801,
        -1.        ],
       [ 0.        ,  0.        ,  0.        , -0.17821783,  0.18001801,
        -1.        ],
       ...,
       [ 0.        ,  0.        ,  0.        ,  0.17731774, -0.17911792,
        -1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.17821783, -0.17911792,
        -1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.17911792, -0.17911792,
        -1.        ]], dtype=float32)

In [289]:
import visu3d as v3d 

idx = 2000

poss = np.concatenate((r3[valid][::idx, :3], r4[valid][::idx, :3]),axis=0)
dirss = np.concatenate((r3[valid][::idx, 3:], r4[valid][::idx, 3:]),axis=0)

# poss = r4[::idx, :3]
# dirss = r4[::idx, 3:]

# poss = np.repeat(sphere_center.reshape(1,3), 160000, 0)
# dirss = sphere_normals
rays = v3d.Ray(pos=poss, dir=dirss)

In [290]:
rays.fig

In [ ]:
import visu3d 
from utils.xyz import rays_single_cam
rays = rays_single_cam([400,400,1111]).T.numpy()


In [ ]:
normals = np.zeros_like(rays)
normals[:,2] =-1

In [ ]:
t = refract(rays, normals, 1/1.5)



In [ ]:
import visu3d as v3d 

origins = np.concatenate((np.zeros_like(rays), rays),axis=0)
dirs = np.concatenate((rays, t),axis=0)
rays_viz = v3d.Ray(pos=origins, dir=dirs)

In [ ]:
rays_viz.fig